<a href="https://colab.research.google.com/github/HarrisonWBlack/Machine-Learning-CAP-5610/blob/master/Assignment%202/HW2_model3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Six layer, dropout, with batch normalization and data augmentation at 25epochs**
This model behaves poorly do to over overfitting.

In [0]:
import matplotlib.pyplot as plt
import keras
import numpy as np
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.layers import Activation, Conv2D, Dense, Dropout, Flatten, MaxPooling2D, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential

(training_images, training_labels), (testing_images, testing_labels) = cifar10.load_data()
batch_size = 256
epochs = 25
augment_data = True
training_labels = keras.utils.to_categorical(training_labels)
testing_labels = keras.utils.to_categorical(testing_labels)


In [0]:
dropout_rate = 0.2

model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=training_images.shape[1:]))
model.add(BatchNormalization())
model.add(MaxPooling2D((2,2)))
model.add(Dropout(dropout_rate))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(dropout_rate))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(dropout_rate))

model.add(Flatten())

model.add(Dense(512, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(2 * dropout_rate))

model.add(Dense(128, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(2 * dropout_rate))

model.add(Dense(10, activation='softmax'))


In [6]:
model.compile(loss="categorical_crossentropy",
             optimizer="Adam",
             metrics=["accuracy"])

training_images = training_images.astype("float32") / 255
testing_images = testing_images.astype("float32") / 255

if not augment_data:
  model.fit(training_images, training_labels,
           batch_size=batch_size,
           epochs=epochs,
           validation_data=(testing_images, testing_labels),
           shuffle=True)
  
else:
  datagen = ImageDataGenerator(
    featurewise_center=False,
    samplewise_center=False,
    featurewise_std_normalization=False,
    samplewise_std_normalization=False,
    zca_whitening=False,
    zca_epsilon=1e-06,
    rotation_range=0,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.,
    zoom_range=0.,
    channel_shift_range=0.,
    fill_mode="nearest",
    cval=0.,
    horizontal_flip=True,
    vertical_flip=False,
    rescale=None,
    preprocessing_function=None,
    data_format=None,
    validation_split=0.0)
  
  datagen.fit(training_images)
  model.fit_generator(datagen.flow(training_images, training_labels,
                                  batch_size=batch_size),
                     epochs=epochs,
                     validation_data=(testing_images, testing_labels),
                     workers=1)
  

Epoch 1/25
196/196 [==============================] - 53s 271ms/step - loss: 1.9939 - acc: 0.3240 - val_loss: 3.1261 - val_acc: 0.1058
Epoch 2/25
196/196 [==============================] - 53s 271ms/step - loss: 1.5247 - acc: 0.4441 - val_loss: 2.2333 - val_acc: 0.2649
Epoch 3/25
196/196 [==============================] - 53s 271ms/step - loss: 1.3566 - acc: 0.5106 - val_loss: 1.1964 - val_acc: 0.5765
Epoch 4/25
196/196 [==============================] - 53s 268ms/step - loss: 1.2615 - acc: 0.5472 - val_loss: 1.2049 - val_acc: 0.5716
Epoch 5/25
196/196 [==============================] - 52s 266ms/step - loss: 1.1919 - acc: 0.5766 - val_loss: 1.0168 - val_acc: 0.6317
Epoch 6/25
196/196 [==============================] - 51s 261ms/step - loss: 1.1286 - acc: 0.5981 - val_loss: 1.0323 - val_acc: 0.6388
Epoch 7/25
196/196 [==============================] - 51s 260ms/step - loss: 1.0801 - acc: 0.6175 - val_loss: 0.9581 - val_acc: 0.6550
Epoch 8/25
196/196 [==============================] - 5

Decent model.

In [7]:
scores = model.evaluate(testing_images, testing_labels, verbose=1)
print("Test loss: ", scores[0])
print("Test accuracy: ", scores[1])

10000/10000 [==============================] - 2s 156us/sample - loss: 0.7626 - acc: 0.7394
Test loss:  0.7625530331611633
Test accuracy:  0.7394
